<a href="https://colab.research.google.com/github/AfsanehHabibi/reddit-conversation-quality/blob/main/Reasoning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
!pip install anytree

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.9/44.9 kB 1.4 MB/s eta 0:00:00


In [21]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [22]:
base_path = "/content/drive/MyDrive/University/RedditData/"

In [23]:
import pickle

# Reading from a file using Pickle
with open(f"{base_path}conversations_with_reasoning.pkl", 'rb') as file:
    conversations = pickle.load(file)

# Print the loaded variables
print("Len conversations:", len(conversations))

Len conversations: 32990


In [24]:
import spacy

nlp = spacy.load("en_core_web_sm")

def contains_reasoning(text):
    doc = nlp(text)

    # Check for specific part-of-speech patterns indicating reasoning
    reasoning_patterns = ["because", "since", "therefore", "due to", "as a result", "consequently", "thus",
                          "for this reason", "in conclusion", "owing to", "on account of", "resulting in",
                          "so", "hence", "in light of", "accordingly", "on the grounds that"]

    for token in doc:
        if token.text.lower() in reasoning_patterns:
            return True

    return False

In [25]:
VERBOSE = 3000

In [26]:
import pickle

def write_conversations_with_reasoning():
  with open(f"{base_path}conversations_reasoning.pkl", 'wb') as file:
    pickle.dump(conversations, file)
  print("Variables written to the file.")

In [27]:
def extract_comments(conversations, num_break_points):
  flat_comments_list = []
  break_points_index = []
  counter = 0
  break_length = int(len(conversations)/num_break_points)
  print("break len ", break_length)
  for id in conversations:
    if counter % break_length == 0:
      break_points_index.append(len(flat_comments_list))
    counter += 1
    comments = conversations[id]['comments']
    for comment in comments:
      flat_comments_list.append(comment)

  return flat_comments_list, break_points_index

In [28]:
import pickle

def write_reasoning_dic(reasoning_dic, file_path):
  with open(file_path, 'wb') as file:
      pickle.dump(reasoning_dic, file)
  print("Variables written to the file.")

In [29]:
import pickle

def merge_conversations_with_reasoning(conversations, file_pathes, method):
  reasoning_dic = dict()
  for file_path in file_pathes:
    with open(file_path, 'rb') as file:
      reasoning_part = pickle.load(file)
      reasoning_dic.update(reasoning_part)

  for id in conversations:
    comments = conversations[id]['comments']
    for comment in comments:
      if not(comment['body'] == '[deleted]' or comment['body'] == '[removed]'):
        if method != "flasher":
          comment['has_reasoning'+method] = reasoning_dic[comment['id']]
        else:
          comment['has_reasoning'] = reasoning_dic[comment['id']]
  with open(f"{base_path}conversations_with_reasoning.pkl", 'wb') as file:
    pickle.dump(conversations, file)
  print("Variables written to the file.")

In [31]:
def calculate_reasoning_for_corpus(corpus, method):
  reasoning_dic = dict()
  for comment in corpus:
      if not(comment['body'] == '[deleted]' or comment['body'] == '[removed]'):
          if method == "flasher":
            has_reasoning = contains_reasoning(comment['body'])
          elif "one-shot":
            has_reasoning = reasoning_based_on_one_shot_classification(comment['body'])
          reasoning_dic[comment['id']] = has_reasoning
  return reasoning_dic

In [32]:
import os

def calculate_reasoning(conversations, num_of_parts, rewrite, method):
  flat_comments, break_points_index = extract_comments(conversations, num_of_parts)
  print("len ", len(break_points_index))
  break_points_index.append(len(flat_comments))
  file_pathes = []
  for i in range(1, len(break_points_index)):
    print("range ", break_points_index[i-1], break_points_index[i])
    file_path = f"{base_path}comments_reasoning_{break_points_index[i-1]}_{break_points_index[i]}.pkl"
    if method != "flasher":
      file_path = f"{base_path}comments_{method}_reasoning_{break_points_index[i-1]}_{break_points_index[i]}.pkl"
    file_pathes.append(file_path)
    print("path ", file_path)
    if rewrite or not os.path.exists(file_path):
      print("do")
      corpus = flat_comments[break_points_index[i-1]:break_points_index[i]]
      reasoning_dic = calculate_reasoning_for_corpus(corpus, method)
      write_reasoning_dic(reasoning_dic, file_path)
  merge_conversations_with_reasoning(conversations, file_pathes, method)

In [34]:
calculate_reasoning(conversations, 10, False, "flasher")
write_conversations_with_reasoning()

break len  3299
len  10
range  0 53506
path  /content/drive/MyDrive/University/RedditData/comments_reasoning_0_53506.pkl
range  53506 116967
path  /content/drive/MyDrive/University/RedditData/comments_reasoning_53506_116967.pkl
range  116967 178801
path  /content/drive/MyDrive/University/RedditData/comments_reasoning_116967_178801.pkl
range  178801 234319
path  /content/drive/MyDrive/University/RedditData/comments_reasoning_178801_234319.pkl
range  234319 300115
path  /content/drive/MyDrive/University/RedditData/comments_reasoning_234319_300115.pkl
range  300115 357006
path  /content/drive/MyDrive/University/RedditData/comments_reasoning_300115_357006.pkl
range  357006 424960
path  /content/drive/MyDrive/University/RedditData/comments_reasoning_357006_424960.pkl
range  424960 478226
path  /content/drive/MyDrive/University/RedditData/comments_reasoning_424960_478226.pkl
range  478226 533555
path  /content/drive/MyDrive/University/RedditData/comments_reasoning_478226_533555.pkl
range  53

In [ ]:
calculate_reasoning(conversations, 10, False, )
write_conversations_with_reasoning("one-shot")

In [ ]:
from transformers import pipeline

# This model is a `zero-shot-classification` model.
# It will classify text, except you are free to choose any label you might imagine
classifier = pipeline(model="facebook/bart-large-mnli")
classifier(
    "I have a problem with my iphone that needs to be resolved asap!!",
    candidate_labels=["urgent", "not urgent", "phone", "tablet", "computer"],
)

In [ ]:
from transformers import pipeline

def contains_reasoning_pipeline(text):
    # Load the BERT model for text classification
    classifier = pipeline('text-classification')

    # Define a prompt to classify
    prompt = f"Is the following text reasoning about something? {text}"

    # Use the BERT model to classify the prompt
    print(classifier(prompt))

    # Check if the predicted label is positive (indicating reasoning)
    return False


No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


[{'label': 'NEGATIVE', 'score': 0.9996144771575928}]
The text does not contain reasoning.


In [ ]:
from transformers import pipeline

question_answerer = pipeline(task="question-answering")
preds = question_answerer(
    question="What is the reason?",
    context="because of everything you said earlier.",
)
print(
    f"score: {round(preds['score'], 4)}, start: {preds['start']}, end: {preds['end']}, answer: {preds['answer']}"
)

No model was supplied, defaulted to distilbert-base-cased-distilled-squad and revision 626af31 (https://huggingface.co/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.


score: 0.2928, start: 0, end: 38, answer: because of everything you said earlier


In [19]:
from transformers import pipeline
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")
texts = [
    "Djokovic has qualified for his sixth French Open final!",
    "On March 23, 2010, President Obama signed the Affordable Care Act into law, putting in place comprehensive reforms that improve access to affordable health coverage for everyone and protect consumers from abusive insurance company practices",
    "The goal was to show that scientists from various disciplines, diverse cultures and countries at different stages of development could find common ground about the conditions for triggering climate action in the current economic context"
]

labels = ["Sport", "Politics", "Environment"]

for text in texts:
    result = classifier(text, labels, multi_label=False)
    print(f"Text: {text}")
    print("Predicted Labels:")
    for label, score in zip(result["labels"], result["scores"]):
        print(f"- {label}: {score}")
    print()

Text: Djokovic has qualified for his sixth French Open final!
Predicted Labels:
- Sport: 0.858054518699646
- Environment: 0.1092466339468956
- Politics: 0.032698873430490494

Text: On March 23, 2010, President Obama signed the Affordable Care Act into law, putting in place comprehensive reforms that improve access to affordable health coverage for everyone and protect consumers from abusive insurance company practices
Predicted Labels:
- Politics: 0.5848667025566101
- Sport: 0.21482090651988983
- Environment: 0.2003123015165329

Text: The goal was to show that scientists from various disciplines, diverse cultures and countries at different stages of development could find common ground about the conditions for triggering climate action in the current economic context
Predicted Labels:
- Environment: 0.8344601988792419
- Sport: 0.08409008383750916
- Politics: 0.08144965022802353



In [ ]:
texts = [
    "Please stop being a jerk, this was the first that came to me",
    "Please stop being a weirdo, my comment was 20 days ago",
    "I replaced two living trees with glowing mushroom trees - all the way to the bottom! Extra pictures show before, map, and bloodmoon.",
    "Yeah, I'm doing the upgrade refund also, but I mean the q will increase overtime which reduce the income, thats why I had to reset it",
    "Average day on r\\/memes",
    "The father confirmed the pictures so it should be true.",
    "You're in South Africa, the average IQ is 70. Of course there are no signs of intelligent life",
    "They should reaaally take out the 'Book 1'"
]

#labels = ["HaveReason", "DoNotHaveReason"]
#labels = ["HaveReasoning", "DoNotHaveReasoning"]
labels = ['Reasoning', "WithoutReasoning"]

for text in texts:
    result = classifier(text, labels, multi_label=False)
    print(f"Text: {text}")
    print("Predicted Labels:")
    for label, score in zip(result["labels"], result["scores"]):
        print(f"- {label}: {score}")
    print()

Text: Please stop being a jerk, this was the first that came to me
Predicted Labels:
- WithoutReasoning: 0.8761652112007141
- Reasoning: 0.12383478134870529

Text: Please stop being a weirdo, my comment was 20 days ago
Predicted Labels:
- WithoutReasoning: 0.8846184015274048
- Reasoning: 0.1153816282749176

Text: I replaced two living trees with glowing mushroom trees - all the way to the bottom! Extra pictures show before, map, and bloodmoon.
Predicted Labels:
- WithoutReasoning: 0.623677134513855
- Reasoning: 0.376322865486145

Text: Yeah, I'm doing the upgrade refund also, but I mean the q will increase overtime which reduce the income, thats why I had to reset it
Predicted Labels:
- WithoutReasoning: 0.5409942269325256
- Reasoning: 0.45900580286979675

Text: Average day on r\/memes
Predicted Labels:
- WithoutReasoning: 0.7949472069740295
- Reasoning: 0.20505277812480927

Text: The father confirmed the pictures so it should be true.
Predicted Labels:
- Reasoning: 0.7656868100166321


In [15]:
!pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 10.2 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=c0d50f63f7ae58591edd1bcfa0333c69fbff74905528cebc24aaffaba5f58b70
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [18]:
from sentence_transformers import SentenceTransformer, util

# Load the pre-trained model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Define two example sentences
sentence1 = "That is a happy person."
sentence2 = "That is a very happy person."

# Encode the sentences into embeddings
embedding1 = model.encode(sentence1, convert_to_tensor=True)
embedding2 = model.encode(sentence2, convert_to_tensor=True)

# Calculate cosine similarity between the embeddings
cosine_similarity = util.pytorch_cos_sim(embedding1, embedding2)

# Print the similarity score
print("Cosine Similarity:", cosine_similarity.item())


Cosine Similarity: 0.9474425315856934
